In [1]:
import cv2
import numpy as np
import open3d as o3d

In [3]:
def getProjected(depth,P):
    
    projected_points = []
    
    print(depth.shape)
    print(P.shape)
    print(np.linalg.inv(P)[:3, :])
    x, y, z = depth.shape
    
    idx = 0
    for i in range(x):
        for j in range(y):
            points_3D = np.linalg.inv(P)[:, :3] @ np.array([j,i,1])
#             print(X.shape)
            points_3D /= points_3D[3]
            
#             print(idx)
            points_3D *= depth[i][j][0] 
            points_3D = points_3D.T
            projected_points.append(points_3D)
            idx += 1
#         print(i)
        
    print(points_3D.shape)
    r = 1080
    c = 1920
    output = np.zeros((c, r, 3))
    i = 0
    j = 0
    
    for p in range(points_3D.shape[0]):
        output[i][j] = points_3D[p]
        j += 1
        
        if j >= c:
            j = 0
            i += 1
        
    print(output.shape)
    return output        

In [5]:
all_pts = np.array([]).reshape(0,3)
all_colors = np.array([]).reshape(0,3)
for i in range(1, 2):
    image = cv2.imread('../rendered_warehouse/camera_capture/images/Image000{}.png'.format(i))
    depth = cv2.imread('../rendered_warehouse/camera_capture/depth/000{}.png'.format(i))


    colors = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    P = np.array([[1,0,0,0],
                  [0,1,0,0.44*i],
                  [0,0,1,0],
                  [0,0,0,1]])
    
    projected_points = getProjected(depth,P)
    print(projected_points.shape)
    print(all_pts.shape)
    
    all_pts = np.concatenate((all_pts,projected_points), axis=0)
    all_colors = np.concatenate((all_colors,colors), axis=0)

(1080, 1920, 3)
(4, 4)
[[ 1.    0.    0.    0.  ]
 [ 0.    1.    0.   -0.44]
 [ 0.    0.    1.    0.  ]]


/home/shashwat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/home/shashwat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/shashwat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in multiply


(4,)
(1920, 1080, 3)
(1920, 1080, 3)
(0, 3)


ValueError: all the input arrays must have same number of dimensions

In [ ]:
mask = (
        (-4000 <= all_pts[:, 0]) & (all_pts[:, 0] < 4000) &
        (-4000 <= all_pts[:, 1]) & (all_pts[:, 1] < 4000) & 
        (-2000 <= all_pts[:, 2]) & (all_pts[:, 2] < 2000)
    )

plot_pts = all_pts[mask]
plot_pts[:, 1] *= -1
plot_colors = (all_colors[mask]/255.0).astype(float)

pcd = o3d.geometry.PointCloud()
vis = o3d.visualization.Visualizer()
pcd.points = o3d.utility.Vector3dVector(plot_pts) #numpy_points is your Nx3 cloud
pcd.colors = o3d.utility.Vector3dVector(plot_colors) #numpy_colors is an Nx3 matrix with the corresponding RGB colors

o3d.visualization.draw_geometries([pcd])

In [ ]:
o3d.io.write_point_cloud("point_cloud.pcd",pcd)